## IMPORT LIBRARIES

In [2]:
from diagrams import Diagram, Cluster, Edge
from diagrams.custom import Custom
from diagrams.aws.storage import S3
from diagrams.programming.framework import FastAPI
from diagrams.onprem.database import Mongodb
from diagrams.saas.analytics import Snowflake
from diagrams.onprem.workflow import Airflow
from diagrams.programming.language import Python

In [1]:
# Adjust these Graphviz attributes to increase diagram size, node spacing, etc.
graph_attr = {
    "fontsize" : "16"
}

filename = "flow_diagram"

## FLOW DIAGRAM CREATION

In [3]:
def create_flow_diagram():
    try:
        with Diagram("Flow Diagram", filename=filename, show=False, direction="LR", graph_attr=graph_attr):
                
                linkedin = Custom("LinkedIn", "./input_icons/linkedin.png")
                indeed = Custom("Indeed", "./input_icons/indeed.png")
                simplyhired = Custom("SimplyHired", "./input_icons/simplyhired.png")
                s3 = S3("Staging")
                mongo = Mongodb("Mongo")
                streamlit = Custom("UI", "./input_icons/streamlit.png")
                fastapi = FastAPI("Fast API")
                user = Custom("User", "./input_icons/user.png")
                
                with Cluster("Resume processing"):
                    text_extract = Python("Resume\ntext extract")
                    vector_resume = Python("Vectorisation")
                    top_jobs = Python("Top 10 jobs")
                
                with Cluster("Daily pipleine"):
                    # airflow1 = Airflow(length="0.5", width="0.5")
                    job_scrape = Python("Job scrape")
                    data_validation = Python("Data Validation")
                    vectorize = Python("Embedding")
                    job_load = Python("Jobs load")
                
                pinecone = Custom("Pinecone", "./input_icons/pinecone.png")
                snowflake = Snowflake("Snowflake")
                huggingface = Custom("Huggingface\nTransformers", "./input_icons/huggingface.png")
                    
                linkedin >> job_scrape
                indeed >> job_scrape
                simplyhired >> job_scrape
                job_scrape >> data_validation >> vectorize >> job_load
                job_scrape >> s3 >> job_load
                data_validation >> Edge(label="") << s3 >> vectorize
                job_load >> snowflake >> top_jobs
                vectorize >> Edge(label="embedding") >> huggingface  >> pinecone >> top_jobs
                
                user >> Edge(label="Uploads Resume") >> streamlit >> fastapi >> Edge(label="User Authentication") << mongo
                streamlit >> Edge(label="Top 10 jobs") >> user
                fastapi >> Edge(label="Raw PDF\n(Resume)") >> s3
                fastapi >> text_extract >>vector_resume >>Edge(label="Pinecone metadata") >> top_jobs
                text_extract >> Edge(label="") << s3
                s3 >> Edge(label="user portfolio") >> vector_resume >> Edge(label="Embedding") >> huggingface
                
                
      
    except Exception as e:
        print("Exception: ",e)
    

In [4]:
create_flow_diagram()